# Подбор гиперпараметров для RandomForest, CatBoost, XGBosst

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as mp
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
import optuna

Подготовка данных

In [ ]:
train_df = pd.read_csv('../data/train_preprocessed.csv')
test_df = pd.read_csv('../data/test_preprocessed.csv')

X_train = train_df.drop('Churn', axis=1)
y_train = train_df['Churn']
X_test = test_df.drop('Churn', axis=1)
y_test = test_df['Churn']

scaler = StandardScaler()

pipeline = mp(
    scaler,
    SMOTE(random_state=42)
)

X_train_processed, y_train_processed = pipeline.fit_resample(X_train, y_train)

X_test_processed = scaler.transform(X_test)

Использование RandomiseSearchCV для RF

In [ ]:
scoring = ['accuracy', 'f1', 'recall', 'roc_auc']

params_rand_rf = {
    'n_estimators': np.random.randint(100, 500),
    'max_depth': np.random.randint(3, 20),
    'min_sample_split': np.random.randint(1, 20),
    'max_sample_split': np.random.randint(2, 20),
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy', 'log_loss']
}

rf = RandomForestClassifier(random_state=42)



rf_rand_search = RandomizedSearchCV(
    rf,
    params_rand_rf,
    n_iter=50,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    random_state=42
)